In [1]:
import pandas as pd
from pandas import DataFrame
df_tennis = DataFrame.from_csv("C:\\Users\\harsh\\ml\\data\\f1.csv")
df_tennis

c:\users\harsh\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


,Outlook,Temperature,Humidity,Wind,Play Tennis
Day,,,,,
d1,sunny,hot,high,weak,no
d2,sunny,hot,high,strong,no
d3,overcast,hot,high,weak,yes
d4,rain,mild,high,weak,yes
d5,rain,cool,normal,weak,yes
d6,rain,cool,normal,strong,no
d7,overcast,cool,normal,strong,yes
d8,sunny,mild,high,weak,no
d9,sunny,cool,normal,weak,yes


In [2]:
def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2) for prob in probs])

def entropy_of_list(a_list):
    from collections import Counter
    cnt = Counter(x for x in a_list)
    print("yes and no Classes:",a_list.name,cnt)
    num_instances=len(a_list)*1
    probs = [x/num_instances for x in cnt.values()]
    return entropy(probs)
total_entropy = entropy_of_list(df_tennis['Play Tennis'])
print("Entropy of given play tennis data set:",total_entropy)

yes and no Classes: Play Tennis Counter({'yes': 9, 'no': 5})
Entropy of given play tennis data set: 0.9402859586706309


In [3]:
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Information Gain Calculation of ",split_attribute_name)

    df_split = df.groupby(split_attribute_name)

    for name,group in df_split: 
        print(name)
        print(group)


    nobs = len(df.index) * 1.0

    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]

    df_agg_ent.columns = ['Entropy', 'PropObservations']



    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy

print('Info-gain for Outlook is :'+str( information_gain(df_tennis, 'Outlook', 'Play Tennis')),"\n")
print('\n Info-gain for Humidity is: ' + str( information_gain(df_tennis, 'Humidity', 'Play Tennis')),"\n")
print('\n Info-gain for Wind is:' + str( information_gain(df_tennis, 'Wind', 'Play Tennis')),"\n")
print('\n Info-gain for Temperature is:' + str( information_gain(df_tennis, 'Temperature','Play Tennis')),"\n")

Information Gain Calculation of  Outlook
overcast
      Outlook Temperature Humidity    Wind Play Tennis
Day                                                   
d3   overcast         hot     high    weak         yes
d7   overcast        cool   normal  strong         yes
d12  overcast        mild     high  strong         yes
d13  overcast         hot   normal    weak         yes
rain
    Outlook Temperature Humidity    Wind Play Tennis
Day                                                 
d4     rain        mild     high    weak         yes
d5     rain        cool   normal    weak         yes
d6     rain        cool   normal  strong          no
d10    rain        mild   normal    weak         yes
d14    rain        mild     high  strong          no
sunny
    Outlook Temperature Humidity    Wind Play Tennis
Day                                                 
d1    sunny         hot     high    weak          no
d2    sunny         hot     high  strong          no
d8    sunny        mild   

In [4]:
def id3(df, target_attribute_name, attribute_names, default_class=None):


    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])


    if len(cnt) == 1:
        return next(iter(cnt))


    elif df.empty or (not attribute_names):
        return default_class


    else:
        default_class = max(cnt.keys())


        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names
                ]
        index_of_max = gainz.index(max(gainz)) 
        best_attr = attribute_names[index_of_max]


        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]


        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

In [5]:
attribute_names = list(df_tennis.columns)
print("List of Attributes:", attribute_names)
attribute_names.remove('Play Tennis') 
print("Predicting Attributes:", attribute_names)

List of Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Tennis']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [6]:
from pprint import pprint
tree = id3(df_tennis,'Play Tennis',attribute_names) 
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)

Information Gain Calculation of  Outlook
overcast
      Outlook Temperature Humidity    Wind Play Tennis
Day                                                   
d3   overcast         hot     high    weak         yes
d7   overcast        cool   normal  strong         yes
d12  overcast        mild     high  strong         yes
d13  overcast         hot   normal    weak         yes
rain
    Outlook Temperature Humidity    Wind Play Tennis
Day                                                 
d4     rain        mild     high    weak         yes
d5     rain        cool   normal    weak         yes
d6     rain        cool   normal  strong          no
d10    rain        mild   normal    weak         yes
d14    rain        mild     high  strong          no
sunny
    Outlook Temperature Humidity    Wind Play Tennis
Day                                                 
d1    sunny         hot     high    weak          no
d2    sunny         hot     high  strong          no
d8    sunny        mild   

In [7]:
def classify(instance, tree, default=None):
    attribute = next(iter(tree))
    if instance[attribute] in tree[attribute].keys():
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict): 
            return classify(instance, result)
        else:
            return result 
    else:
        return default

df_tennis['predicted'] = df_tennis.apply(classify, axis=1, args=(tree,'No') )
print('Accuracy is:' + str( sum(df_tennis['Play Tennis']==df_tennis['predicted'] ) / (1.0*len(df_tennis.index)) ))

df_tennis[['Play Tennis', 'predicted']]

Accuracy is:1.0


,Play Tennis,predicted
Day,,
d1,no,no
d2,no,no
d3,yes,yes
d4,yes,yes
d5,yes,yes
d6,no,no
d7,yes,yes
d8,no,no
d9,yes,yes


In [8]:
training_data = df_tennis.iloc[1:-4]
test_data = df_tennis.iloc[-4:]
train_tree = id3(training_data,'Play Tennis',attribute_names)

test_data['predicted2'] = test_data.apply(classify,
                                         axis=1,
                                         args=(train_tree,'Yes'))
print('\n\n Accuracy is : ' + str( sum(test_data['Play Tennis']==test_data['predicted2'] ) / (1.0*len(test_data.index) )))

Information Gain Calculation of  Outlook
overcast
      Outlook Temperature Humidity    Wind Play Tennis predicted
Day                                                             
d3   overcast         hot     high    weak         yes       yes
d7   overcast        cool   normal  strong         yes       yes
rain
    Outlook Temperature Humidity    Wind Play Tennis predicted
Day                                                           
d4     rain        mild     high    weak         yes       yes
d5     rain        cool   normal    weak         yes       yes
d6     rain        cool   normal  strong          no        no
d10    rain        mild   normal    weak         yes       yes
sunny
    Outlook Temperature Humidity    Wind Play Tennis predicted
Day                                                           
d2    sunny         hot     high  strong          no        no
d8    sunny        mild     high    weak          no        no
d9    sunny        cool   normal    weak         

c:\users\harsh\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
